The method is described in [Intro2Robotics: Connected Components in a Binary Image](https://www.youtube.com/watch?v=ticZclUYy88) and its Python implementation has been illustrated [here](https://zhuanlan.zhihu.com/p/97689424). Basically, we need to interrogate each Px in each row and label them different number if they are not connected and do this row by row. Eventually, we are able to update the graph with an Equ list, which should which components are essentially the same.

In [207]:
import numpy as np
import pandas as pd

In [208]:
RawImage = pd.read_table("input_question_4", header = None) 
# Remove auto-assigned col names
RawImage

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,0,0,1,1,0,0,0,1,0,1,0,1,1,1,0,0,1,1
1,1,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,0
2,0,1,1,1,1,1,1,0,1,0,1,0,1,0,0,0,0,0,1,0
3,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,1,1,0,1,1
4,1,0,0,1,0,0,0,0,0,0,1,1,1,0,1,1,1,0,1,1
5,1,1,1,1,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1
6,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1
7,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,1,1,1
8,0,0,1,1,1,0,0,1,0,1,0,0,1,0,0,0,1,1,1,0
9,1,0,1,0,1,0,1,1,1,1,0,0,1,0,1,0,0,0,0,1


In [209]:
RawImage=np.array(RawImage)

In [210]:
OFFSETS_4 = [[0, -1], [-1, 0], [0, 0], [1, 0], [0, 1]]
OFFSETS_8 = [[-1, -1], [0, -1], [1, -1],
             [-1,  0], [0,  0], [1,  0],
             [-1,  1], [0,  1], [1,  1]]

In [212]:
def neighbor_value(raw_img: np.array, offsets, reverse=False):
    
    rows, cols = raw_img.shape
    label_idx = 0
    
    rows_ = [0, rows, 1] if reverse == False else [rows-1, -1, -1]
    cols_ = [0, cols, 1] if reverse == False else [cols-1, -1, -1]
    
    # Emuration by long and short sides
    for row in range(rows_[0], rows_[1], rows_[2]):
        for col in range(cols_[0], cols_[1], cols_[2]):
            # initial label == 0
            label = 0
            
            # If the point is lit and then continue
            if raw_img[row][col] == 0 :
                continue
                
            for offset in offsets:
                neighbor_row = min(max(0, row+offset[0]), rows-1)
                neighbor_col = min(max(0, col+offset[1]), cols-1)
                neighbor_val = raw_img[neighbor_row, neighbor_col]
                
                if neighbor_val == 0:
                    continue
                    
                label = neighbor_val if neighbor_val < label else label
            if label == 0:
                label_idx += 1
                label = label_idx
            raw_img[row][col] = label
    return raw_img

In [213]:
FirstPass=neighbor_value(RawImage, OFFSETS_4, False)
pd.DataFrame(FirstPass)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,0,0,1,2,0,0,0,3,0,4,0,5,6,7,0,0,8,9
1,10,0,11,0,0,0,0,0,12,13,0,0,0,0,14,0,0,15,0,0
2,0,16,17,18,19,20,21,0,22,0,23,0,24,0,0,0,0,0,25,0
3,0,0,0,0,26,27,0,0,0,0,28,0,0,0,29,30,31,0,32,33
4,34,0,0,35,0,0,0,0,0,0,36,37,38,0,39,40,41,0,42,43
5,44,45,46,47,0,48,0,0,0,0,0,0,0,49,0,0,50,51,0,52
6,53,54,0,0,0,0,55,0,0,0,0,56,57,0,0,0,0,58,0,59
7,0,60,0,0,0,61,62,0,0,0,0,63,64,0,0,0,0,65,66,67
8,0,0,68,69,70,0,0,71,0,72,0,0,73,0,0,0,74,75,76,0
9,77,0,78,0,79,0,80,81,82,83,0,0,84,0,85,0,0,0,0,86


In [206]:
SecondPass=neighbor_value(FirstPass, OFFSETS_4, True)
pd.DataFrame(SecondPass)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0,0,0,0,86,85,0,0,0,84,0,83,0,82,81,80,0,0,79,78
1,77,0,76,0,0,0,0,0,75,74,0,0,0,0,73,0,0,72,0,0
2,0,71,70,69,68,67,66,0,65,0,64,0,63,0,0,0,0,0,62,0
3,0,0,0,0,61,60,0,0,0,0,59,0,0,0,58,57,56,0,55,54
4,53,0,0,52,0,0,0,0,0,0,51,50,49,0,48,47,46,0,45,44
5,43,42,41,40,0,39,0,0,0,0,0,0,0,38,0,0,37,36,0,35
6,34,33,0,0,0,0,32,0,0,0,0,31,30,0,0,0,0,29,0,28
7,0,27,0,0,0,26,25,0,0,0,0,24,23,0,0,0,0,22,21,20
8,0,0,19,18,17,0,0,16,0,15,0,0,14,0,0,0,13,12,11,0
9,10,0,9,0,8,0,7,6,5,4,0,0,3,0,2,0,0,0,0,1
